In [73]:
import sys
sys.path.append('/home/user/dev/kasi-x/akizora/src')

In [96]:
import json
from pathlib import Path

import structlog

from logger_config import configure_logger
from scrayping.github_api import GithubAPI

configure_logger()

logger = structlog.get_logger(__name__).bind(module="file_save")


def save_dict(data: dict, path: Path) -> None:
    if not path.exists():
        path.touch()
    with open(path, "w") as fp:
        json.dump(data, fp)


def read_dict(path):
    with open(path) as fp:
        return json.load(fp)


def save_file(title: str = "john-maynard-keynes_the-economic-consequences-of-the-peace") -> None:
    cached_file_path = Path(f"{title}/info.json")
    if cached_file_path.exists():
        tree_info = read_dict(cached_file_path)
    else:
        github_api = GithubAPI()
        tree_info = github_api.get_tree_info(title)
        save_dict(tree_info, cached_file_path)

    for file_name, file_info in tree_info.items():
        file_content_path = Path(f"{title}/{file_name}.json")
        if not file_content_path.exists():
            try:
                file_content = github_api.process_file_info(file_info)
                print(f"--- {file_name} ---")
            except Exception as e:
                logger.exception("Failed to process file.", file_path=file_name, error=str(e))
            with open(file_content_path, "w") as fp:
                json.dump(file_content, fp)


def main():
    title = "john-maynard-keynes_the-economic-consequences-of-the-peace"
    save_file(title)
    info_dict = read_dict(f"{title}/info.json")
    for file_name in info_dict:
        file_content = read_dict(f"{title}/{file_name}.json")
        print(f"--- {file_name} ---")
        print(file_content)

In [97]:
main()

2024-04-09 22:55.14 [info     ] API Request                    [scrayping.github_api] method=GET module=github_api url=https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:
2024-04-09 22:55.14 [info     ] API Request                    [scrayping.github_api] method=GET module=github_api url=https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:
2024-04-09 22:55.14 [info     ] API Request                    [scrayping.github_api] method=GET module=github_api url=https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:
2024-04-09 22:55.14 [info     ] API Request                    [scrayping.github_api] method=GET module=github_api url=https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:
2024-04-09 22:55.14 [info     ] API Requ

FileNotFoundError: [Errno 2] No such file or directory: 'john-maynard-keynes_the-economic-consequences-of-the-peace/info.json'

In [94]:
def main():
    # url = input("Enter GitHub API URL: ")
    title = "john-maynard-keynes_the-economic-consequences-of-the-peace"
    github_api = GithubAPI()
    tree_info = github_api.get_tree_info(title)
    for file_info in tree_info:
        try:
            github_api.process_file_info(file_info)
            print(f"--- {file_info['path']} ---")
        except Exception as e:
            logger.exception("Failed to process file.", file_path=file_info["path"], error=str(e))

In [95]:
main()

2024-04-09 22:28.23 [info     ] API Request                    [__main__] method=GET url=https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:src/epub/text
2024-04-09 22:28.23 [info     ] API Request                    [__main__] method=GET url=https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:src/epub/text
2024-04-09 22:28.23 [info     ] API Request                    [__main__] method=GET url=https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:src/epub/text
Starting new HTTPS connection (1): api.github.com:443
Starting new HTTPS connection (1): api.github.com:443
Starting new HTTPS connection (1): api.github.com:443
https://api.github.com:443 "GET /repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:src/epub/text HTTP/1.1" 200 None
https:

In [77]:
main()

Starting new HTTPS connection (1): api.github.com:443
Starting new HTTPS connection (1): api.github.com:443
https://api.github.com:443 "GET /repos/standardebooks/the-economic-consequences-of-the-peace/git/trees/master:src/epub/text HTTP/1.1" 404 None
https://api.github.com:443 "GET /repos/standardebooks/the-economic-consequences-of-the-peace/git/trees/master:src/epub/text HTTP/1.1" 404 None


Exception: Failed to get tree info.

In [53]:
import base64
import os
import xml
import xml.etree.ElementTree as ET
from enum import Enum

import requests
import structlog
from lxml import etree
from lxml.etree import XMLParser

# from logger_config import configure_logger
#
# configure_logger()
# logger = structlog.get_logger(__name__)


def get_file_info(url, headers=None):
    """URLを引数にして、ファイルの情報をdictにして返す."""
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        tree_info = response.json()
        file_info_dict: dict[str, bytes] = {}
        for file_info in tree_info["tree"]:
            file_name = file_info["path"]
            file_type = file_info["type"]

            # ファイルの内容を取得
            if file_type != "blob":
                msg = f"Error: {file_type} is not supported."
                raise Exception(msg)

            url = (
                "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/blobs/"
                + file_info["sha"]
            )
            response = requests.get(url)
            if response.status_code == 200:
                file_content = base64.b64decode(response.json()["content"])
                file_info_dict[file_name] = file_content
            else:
                msg = f"Error: {response.status_code}, {response.text} {url}"
                raise Exception(msg)
        return file_info_dict
    else:
        msg = f"Error: {response.status_code}, {response.text} {url}"

        raise Exception(msg)


# 結果を出力
# print(file_info_dict)


class TextComponent:
    def __init__(self, title, sections):
        self.title = title
        self.sections = sections


class SectionComponent:
    def __init__(self, component_type):
        self.type = component_type


class SectionType(Enum):
    SECTION = "section"
    BLOCKQUOTE = "blockquote"


class HgroupInfo:
    def __init__(self, ordinal, title):
        self.ordinal = ordinal
        self.title = title


class Section:
    def __init__(self, sectiontype, section_id, epub_type, hgroup_info, paragraphs):
        self.sectiontype: SectionType = sectiontype
        self.section_id = section_id
        self.epub_type = epub_type
        self.hgroup_info = hgroup_info
        self.paragraphs = paragraphs


def get_page_title(data_root):
    namespaces = {"xhtml": "http://www.w3.org/1999/xhtml"}
    return data_root.xpath("//xhtml:head/xhtml:title", namespaces=namespaces)[0].text


def process_blockquote(blockquote):
    """Blockquote 要素からテキストを抽出して整形します."""
    namespaces = {"xhtml": "http://www.w3.org/1999/xhtml"}

    blocks = []
    for div in blockquote.findall("xhtml:div", namespaces=namespaces):
        header_text = div.find("xhtml:header/xhtml:p", namespaces=namespaces).text
        if header_text:
            blocks.append(f"**{header_text}**")

        # 各span要素からテキストを抽出する
        paragraph = " ".join(
            span.text for span in div.findall("xhtml:p/xhtml:span", namespaces=namespaces)
        )
        blocks.append(paragraph)

    return "\n".join(blocks)


def parse_xhtml(data):
    """XHTMLデータを解析し、TextComponentインスタンスを生成します。.

    Args:
        data: XHTMLデータ

    Returns:
        TextComponentインスタンス
    """
    namespaces = {"xhtml": "http://www.w3.org/1999/xhtml"}

    root = data
    title = get_page_title(root)

    # セクション情報の取得
    sections = []
    for section in root.xpath("//xhtml:body/xhtml:section", namespaces=namespaces):
        # セクションID
        section_id = section.get("id")
        epub_type = section.get("epub:type")
        hgroup_info = HgroupInfo(
            ordinal=section.find("xhtml:h2", namespaces=namespaces).text
            if section.find("xhtml:h2", namespaces=namespaces) is not None
            else None,
            title=section.find("xhtml:p", namespaces=namespaces).text
            if section.find("xhtml:p", namespaces=namespaces) is not None
            else None,
        )

        paragraphs = []  # 通常のp要素も取得
        temporary = []
        # section 内の各要素を順番に処理

        # section 内の各要素を順番に処理
        for element in section:
            if element.tag == "{http://www.w3.org/1999/xhtml}p":
                if temporary:
                    paragraphs.append("-------something------")
                    temporary = []
                paragraphs.append(element.text)
            elif element.tag == "{http://www.w3.org/1999/xhtml}blockquote":
                temporary.append(element)
                #text = process_blockquote(element)
                # paragraphs.append(text)  # 処理後のblockquoteを挿入

        # セクション情報に追加
        section_type = (
            SectionType.SECTION if epub_type != "z3998:verse" else SectionType.BLOCKQUOTE
        )
        sections.append(Section(section_type, section_id, epub_type, hgroup_info, paragraphs))

    # データ構造の生成
    text_component = TextComponent(title, sections)

    return text_component



In [80]:
def get_file_info(url, headers=None):
    """URLを引数にして、ファイルの情報をdictにして返す."""
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        tree_info = response.json()
        file_info_dict: dict[str, bytes] = {}
        for file_info in tree_info["tree"]:
            file_name = file_info["path"]
            file_type = file_info["type"]

            # ファイルの内容を取得
            if file_type != "blob":
                msg = f"Error: {file_type} is not supported."
                raise Exception(msg)

            url = (
                "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/blobs/"
                + file_info["sha"]
            )
            response = requests.get(url)
            if response.status_code == 200:
                file_content = base64.b64decode(response.json()["content"])
                file_info_dict[file_name] = file_content
            else:
                msg = f"Error: {response.status_code}, {response.text} {url}"
                raise Exception(msg)
        return file_info_dict
    else:
        msg = f"Error: {response.status_code}, {response.text} {url}"

        raise Exception(msg)

get_file_info(url = "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:src/epub/text")

In [82]:
get_file_info(url = "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:src/epub/text")

Starting new HTTPS connection (1): api.github.com:443
Starting new HTTPS connection (1): api.github.com:443
https://api.github.com:443 "GET /repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:src/epub/text HTTP/1.1" 200 825
https://api.github.com:443 "GET /repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:src/epub/text HTTP/1.1" 200 825
Starting new HTTPS connection (1): api.github.com:443
Starting new HTTPS connection (1): api.github.com:443
https://api.github.com:443 "GET /repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/blobs/5a6be444aff865b840702deeec4ef6c2cdeacf87 HTTP/1.1" 200 None
https://api.github.com:443 "GET /repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/blobs/5a6be444aff865b840702deeec4ef6c2cdeacf87 HTTP/1.1" 200 None
Starting new HTTPS connection (1): api.github.com:443
Starting new HTTPS connection (1): api.g

{'chapter-1.xhtml': b'<?xml version="1.0" encoding="utf-8"?>\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:epub="http://www.idpf.org/2007/ops" epub:prefix="z3998: http://www.daisy.org/z3998/2012/vocab/structure/, se: https://standardebooks.org/vocab/1.0" xml:lang="en-GB">\n\t<head>\n\t\t<title>I: Introductory</title>\n\t\t<link href="../css/core.css" rel="stylesheet" type="text/css"/>\n\t\t<link href="../css/local.css" rel="stylesheet" type="text/css"/>\n\t</head>\n\t<body epub:type="bodymatter z3998:fiction">\n\t\t<section id="chapter-1" epub:type="chapter">\n\t\t\t<hgroup>\n\t\t\t\t<h2 epub:type="ordinal z3998:roman">I</h2>\n\t\t\t\t<p epub:type="title">Introductory</p>\n\t\t\t</hgroup>\n\t\t\t<p>The power to become habituated to his surroundings is a marked characteristic of mankind. Very few of us realize with conviction the intensely unusual, unstable, complicated, unreliable, temporary nature of the economic organization by which Western Europe has lived for the last half cen

In [27]:
namespaces = {"xhtml": "http://www.w3.org/1999/xhtml"}

In [28]:
raw_section = root.xpath("//xhtml:body/xhtml:section", namespaces=namespaces)
raw_section[0].get("blockquote")

In [35]:
blockquote = raw_section[0].find("{http://www.w3.org/1999/xhtml}blockquote")

In [43]:
type(raw_section[0])

lxml.etree._Element

In [52]:
# blockquote要素を除いた兄弟要素すべてを取得
if blockquote is not None:
    for element in blockquote.itersiblings():
        print(element.tag, element.text)

{http://www.w3.org/1999/xhtml}p In Paris, where those connected with the Supreme Economic Council, received almost hourly the reports of the misery, disorder, and decaying organization of all Central and Eastern Europe, allied and enemy alike, and learnt from the lips of the financial representatives of Germany and Austria unanswerable evidence, of the terrible exhaustion of their countries, an occasional visit to the hot, dry room in the President’s house, where the Four fulfilled their destinies in empty and arid intrigue, only added to the sense of nightmare. Yet there in Paris the problems of Europe were terrible and clamant, and an occasional return to the vast unconcern of London a little disconcerting. For in London these questions were very far away, and our own lesser problems alone troubling. London believed that Paris was making a great confusion of its business, but remained uninterested. In this spirit the British people received the Treaty without reading it. But it is un

In [44]:
# blockquote要素内の子要素すべてを取得
if blockquote is not None:
    elements = list(blockquote.iter())  # list()関数で要素をリストに変換
    for element in elements:
        print(element.tag, element.text)

{http://www.w3.org/1999/xhtml}blockquote 
				
{http://www.w3.org/1999/xhtml}div 
					
{http://www.w3.org/1999/xhtml}header 
						
{http://www.w3.org/1999/xhtml}p Spirit of the Years
{http://www.w3.org/1999/xhtml}p 
						
{http://www.w3.org/1999/xhtml}span Observe that all wide sight and self-command
{http://www.w3.org/1999/xhtml}br None
{http://www.w3.org/1999/xhtml}span Deserts these throngs now driven to demonry
{http://www.w3.org/1999/xhtml}br None
{http://www.w3.org/1999/xhtml}span By the Immanent Unrecking. Nought remains
{http://www.w3.org/1999/xhtml}br None
{http://www.w3.org/1999/xhtml}span But vindictiveness here amid the strong,
{http://www.w3.org/1999/xhtml}br None
{http://www.w3.org/1999/xhtml}span And there amid the weak an impotent rage.
{http://www.w3.org/1999/xhtml}div 
					
{http://www.w3.org/1999/xhtml}header 
						
{http://www.w3.org/1999/xhtml}p Spirit of the Pities
{http://www.w3.org/1999/xhtml}p 
						
{http://www.w3.org/1999/xhtml}span Why prompts the Will

In [2]:
api_token = os.environ["GITHUB_PERSONAL_ACCESS_TOKEN"]
url = "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:src/epub/text"
headers = {"Authorization": "token %s" % api_token}
file_info_dict = get_file_info(url, headers)

In [59]:
namespaces = {"xhtml": "http://www.w3.org/1999/xhtml"}

title = get_page_title(root)

# セクション情報の取得
sections = []
for section in root.xpath("//xhtml:body/xhtml:section", namespaces=namespaces):
    # セクションID
    section_id = section.get("id")
    epub_type = section.get("epub:type")
    hgroup_info = HgroupInfo(
        ordinal=section.find("xhtml:h2", namespaces=namespaces).text
        if section.find("xhtml:h2", namespaces=namespaces) is not None
        else None,
        title=section.find("xhtml:p", namespaces=namespaces).text
        if section.find("xhtml:p", namespaces=namespaces) is not None
        else None,
    )

    paragraphs = []  # 通常のp要素も取得
    temporary = []
    target = []
    # section 内の各要素を順番に処理

    # section 内の各要素を順番に処理
    for element in section:
        if element.tag == "{http://www.w3.org/1999/xhtml}p":
            if temporary:
                paragraphs.append("-------something------")
                target = temporary.copy()
                temporary = []
            paragraphs.append(element.text)
        elif element.tag == "{http://www.w3.org/1999/xhtml}blockquote":
            temporary.append(element)
            #text = process_blockquote(element)
            # paragraphs.append(text)  # 処理後のblockquoteを挿入

    # セクション情報に追加
    section_type = (
        SectionType.SECTION if epub_type != "z3998:verse" else SectionType.BLOCKQUOTE
    )
    sections.append(Section(section_type, section_id, epub_type, hgroup_info, paragraphs))

# データ構造の生成
text_component = TextComponent(title, sections)

In [67]:
blockquote = target[0]

In [68]:
# blockquote要素内の子要素すべてを取得
if blockquote is not None:
    elements = list(blockquote.iter())  # list()関数で要素をリストに変換
    for element in elements:
        print(element.tag, element.text)

{http://www.w3.org/1999/xhtml}blockquote 
				
{http://www.w3.org/1999/xhtml}div 
					
{http://www.w3.org/1999/xhtml}header 
						
{http://www.w3.org/1999/xhtml}p Spirit of the Years
{http://www.w3.org/1999/xhtml}p 
						
{http://www.w3.org/1999/xhtml}span Observe that all wide sight and self-command
{http://www.w3.org/1999/xhtml}br None
{http://www.w3.org/1999/xhtml}span Deserts these throngs now driven to demonry
{http://www.w3.org/1999/xhtml}br None
{http://www.w3.org/1999/xhtml}span By the Immanent Unrecking. Nought remains
{http://www.w3.org/1999/xhtml}br None
{http://www.w3.org/1999/xhtml}span But vindictiveness here amid the strong,
{http://www.w3.org/1999/xhtml}br None
{http://www.w3.org/1999/xhtml}span And there amid the weak an impotent rage.
{http://www.w3.org/1999/xhtml}div 
					
{http://www.w3.org/1999/xhtml}header 
						
{http://www.w3.org/1999/xhtml}p Spirit of the Pities
{http://www.w3.org/1999/xhtml}p 
						
{http://www.w3.org/1999/xhtml}span Why prompts the Will

In [54]:
xml_string = file_info_dict.get("chapter-1.xhtml")
parser = XMLParser(
    encoding="UTF-8", resolve_entities=xFalse, strip_cdata=False, recover=True, ns_clean=True
)
root = ET.fromstring(xml_string, parser)
a = parse_xhtml(root)

#pretty_xml = etree.tostring(root, pretty_print=True, encoding=str)
#print(pretty_xml)

In [64]:
#a.sections[0].paragraphs

In [8]:
len(a.sections)

1

In [6]:
for i in a.sections:
    print(i.sectiontype)

SectionType.SECTION


In [17]:
import base64
import os
import xml
import xml.etree.ElementTree as ET

import requests
import structlog
from lxml import etree
from lxml.etree import XMLParser

# from logger_config import configure_logger
#
# configure_logger()
# logger = structlog.get_logger(__name__)


def get_file_info(url, headers=None):
    """URLを引数にして、ファイルの情報をdictにして返す."""
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        tree_info = response.json()
        file_info_dict: dict[str, bytes] = {}
        for file_info in tree_info["tree"]:
            file_name = file_info["path"]
            file_type = file_info["type"]

            # ファイルの内容を取得
            if file_type != "blob":
                msg = f"Error: {file_type} is not supported."
                raise Exception(msg)

            url = (
                "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/blobs/"
                + file_info["sha"]
            )
            response = requests.get(url)
            if response.status_code == 200:
                file_content = base64.b64decode(response.json()["content"])
                file_info_dict[file_name] = file_content
            else:
                msg = f"Error: {response.status_code}, {response.text} {url}"
                raise Exception(msg)
        return file_info_dict
    else:
        msg = f"Error: {response.status_code}, {response.text} {url}"

        raise Exception(msg)


# 結果を出力
# print(file_info_dict)


class TextComponent:
    def __init__(self, title, sections):
        self.title = title
        self.sections = sections


class Section:
    def __init__(self, section_id, epub_type, hgroup_info, paragraphs):
        self.section_id = section_id
        self.epub_type = epub_type
        self.hgroup_info = hgroup_info
        self.paragraphs = paragraphs


class HgroupInfo:
    def __init__(self, ordinal, title):
        self.ordinal = ordinal
        self.title = title


def get_page_title(data_root):
    namespaces = {"xhtml": "http://www.w3.org/1999/xhtml"}
    return data_root.xpath("//xhtml:head/xhtml:title", namespaces=namespaces)[0].text


def parse_xhtml(data):
    """XHTMLデータを解析し、TextComponentインスタンスを生成します。.

    Args:
        data: XHTMLデータ

    Returns:
        TextComponentインスタンス
    """
    namespaces = {"xhtml": "http://www.w3.org/1999/xhtml"}

    root = data
    title = get_page_title(root)

    # セクション情報の取得
    sections = []
    for section in root.xpath("//xhtml:body/xhtml:section", namespaces=namespaces):
        # セクションID
        section_id = section.get("id")

                # hgroup情報の取得
        # hgroup情報の取得
        hgroup = section.find("xhtml:hgroup", namespaces=namespaces)
        if hgroup is not None:
            hgroup_info = HgroupInfo(
                ordinal=hgroup.find("xhtml:h2[@epub:type='ordinal']", namespaces=namespaces).text if hgroup.find("xhtml:h2[@epub:type='ordinal']", namespaces=namespaces) else None,
                title=hgroup.find("xhtml:p[@epub:type='title']", namespaces=namespaces).text if hgroup.find("xhtml:p[@epub:type='title']", namespaces=namespaces) else None,
            )
        else:
            hgroup_info = None

        # p要素の内容を取得
        paragraphs = []
        for paragraph in section.findall("xhtml:p", namespaces=namespaces):
            paragraphs.append(paragraph.text)

        # セクション情報に追加
        sections.append(Section(section_id, section.get("epub:type"), hgroup_info, paragraphs))

    # データ構造の生成
    text_component = TextComponent(title, sections)

    return text_component



In [18]:
api_token = os.environ["GITHUB_PERSONAL_ACCESS_TOKEN"]
url = "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:src/epub/text"
headers = {"Authorization": "token %s" % api_token}
file_info_dict = get_file_info(url, headers)
xml_string = file_info_dict.get("chapter-1.xhtml")
parser = XMLParser(
    encoding="UTF-8", resolve_entities=False, strip_cdata=False, recover=True, ns_clean=True
)
root = ET.fromstring(xml_string, parser)


#pretty_xml = etree.tostring(root, pretty_print=True, encoding=str)
#print(pretty_xml)

In [21]:
root

<Element {http://www.w3.org/1999/xhtml}html at 0x7fbf865b6dc0>

In [19]:
pretty_xml = etree.tostring(root, pretty_print=True, encoding=str)

In [24]:
a = parse_xhtml(root)

In [22]:
for b in a.sections:
    c = "".join(b.paragraphs)
    if "sprit" in c:
        print(c)

In [ ]:
text_com

In [ ]:
file_info_dict.keys()

In [22]:
xml_string = file_info_dict.get("chapter-1.xhtml")

In [ ]:
# file_info_dict.keys()

In [ ]:
# xml_string = base64.b64decode(content_raw)

In [25]:
import xml.etree.ElementTree as ET
from lxml.etree import XMLParser

parser = XMLParser(encoding="UTF-8", resolve_entities=False, strip_cdata=False, recover=True, ns_clean=True)
root = ET.fromstring(xml_string, parser)
pretty_xml = lxml.etree.tostring(root, pretty_print=True, encoding=str)

NameError: name 'lxml' is not defined

In [ ]:
pretty_xml = lxml.etree.tostring(root, pretty_print=True, encoding=str)

In [ ]:
import xml.dom.minidom

document = 

dom = xml.dom.minidom.parseString(document)

In [ ]:
import xml.dom.minidom

document = """\
<slideshow>
<title>Demo slideshow</title>
<slide><title>Slide title</title>
<point>This is a demo</point>
<point>Of a program for processing slides</point>
</slide>

<slide><title>Another demo slide</title>
<point>It is important</point>
<point>To have more than</point>
<point>one slide</point>
</slide>
</slideshow>
"""

dom = xml.dom.minidom.parseString(document)

In [ ]:
def getText(nodelist):
    rc = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
    return ''.join(rc)

def handleSlideshow(slideshow):
    print("<html>")
    handleSlideshowTitle(slideshow.getElementsByTagName("title")[0])
    slides = slideshow.getElementsByTagName("slide")
    handleToc(slides)
    handleSlides(slides)
    print("</html>")

def handleSlides(slides):
    for slide in slides:
        handleSlide(slide)

def handleSlide(slide):
    handleSlideTitle(slide.getElementsByTagName("title")[0])
    handlePoints(slide.getElementsByTagName("point"))

def handleSlideshowTitle(title):
    print(f"<title>{getText(title.childNodes)}</title>")

def handleSlideTitle(title):
    print(f"<h2>{getText(title.childNodes)}</h2>")

def handlePoints(points):
    print("<ul>")
    for point in points:
        handlePoint(point)
    print("</ul>")

def handlePoint(point):
    print(f"<li>{getText(point.childNodes)}</li>")

def handleToc(slides):
    for slide in slides:
        title = slide.getElementsByTagName("title")[0]
        print(f"<p>{getText(title.childNodes)}</p>")

handleSlideshow(dom)

In [ ]:
print(pretty_xml[:1000])

In [ ]:
data = pretty_xml

In [ ]:
from lxml import etree


namespaces = {'xhtml': 'http://www.w3.org/1999/xhtml'}
#root = pretty_xml
# タイトル取得
title = root.xpath("//xhtml:head/xhtml:title", namespaces=namespaces)[0].text

# セクション情報の取得
section_info = {}
for section in root.xpath("//xhtml:body/xhtml:section", namespaces=namespaces):
    # セクションID
    section_id = section.get("id")
    
    # hgroup情報の取得
    hgroup_info = {}
    hgroup = section.find("xhtml:hgroup", namespaces=namespaces)
    if hgroup is not None:
        # 章番号
        hgroup_info["ordinal"] = hgroup.find("xhtml:h2", namespaces=namespaces).text
        # タイトル
        hgroup_info["title"] = hgroup.find("xhtml:p", namespaces=namespaces).text
    
    # p要素の内容を取得
    paragraphs = []
    for paragraph in section.findall("xhtml:p", namespaces=namespaces):
        paragraphs.append(paragraph.text)
    
    # セクション情報に追加
    section_info[section_id] = hgroup_info, paragraphs

# 結果の出力
print(f"タイトル: {title}")
for section_id, (hgroup_info, paragraphs) in section_info.items():
    print(f"\nセクションID: {section_id} ({section.get('epub:type')})")
    print(f"  章番号: {hgroup_info['ordinal']}")
    print(f"  タイトル: {hgroup_info['title']}")
    print(f"  本文:")
    for paragraph in paragraphs:
        print(f"    {paragraph}")

In [ ]:
import xml

In [ ]:
class TextComponent:
    def __init__(self, title, sections):
        self.title = title
        self.sections = sections

class Section:
    def __init__(self, section_id, epub_type, hgroup_info, paragraphs):
        self.section_id = section_id
        self.epub_type = epub_type
        self.hgroup_info = hgroup_info
        self.paragraphs = paragraphs

class HgroupInfo:
    def __init__(self, ordinal, title):
        self.ordinal = ordinal
        self.title = title

In [ ]:
def parse_xhtml(data):
    """
    XHTMLデータを解析し、TextComponentインスタンスを生成します。

    Args:
        data: XHTMLデータ

    Returns:
        TextComponentインスタンス
    """

    # ネームスペースを定義
    namespaces = {'xhtml': 'http://www.w3.org/1999/xhtml'}

    # ルート要素を取得
    root = xml.etree.fromstring(data, nsmap=namespaces)

    # タイトル取得
    title = root.xpath("//xhtml:head/xhtml:title", namespaces=namespaces)[0].text

    # セクション情報の取得
    sections = []
    for section in root.xpath("//xhtml:body/xhtml:section", namespaces=namespaces):
        # セクションID
        section_id = section.get("id")

        # hgroup情報の取得
        hgroup_info = HgroupInfo(
            ordinal=section.find("xhtml:h2", namespaces=namespaces).text if section.find("xhtml:h2", namespaces=namespaces) is not None else None,
            title=section.find("xhtml:p", namespaces=namespaces).text if section.find("xhtml:p", namespaces=namespaces) is not None else None,
        )

        # p要素の内容を取得
        paragraphs = []
        for paragraph in section.findall("xhtml:p", namespaces=namespaces):
            paragraphs.append(paragraph.text)

        # セクション情報に追加
        sections.append(Section(section_id, section.get("epub:type"), hgroup_info, paragraphs))

    # データ構造の生成
    text_component = TextComponent(title, sections)

    return text_component

In [ ]:
a = parse_xhtml(root)

In [ ]:
text_component.sections[0].paragraphs

In [ ]:
title_element = root.xpath("//xhtml:head/xhtml:title", namespaces=namespaces)[0]

In [ ]:
title_element.text

In [ ]:
a = root.xpath('//xhtml:section//xhtml:p', namespaces=namespaces)

In [ ]:
a[0]

In [ ]:
for i in a:
    print(i.text)

In [ ]:
a.text

In [ ]:
title_element = html.xpath("/xhtml:head/xhtml:title")

In [ ]:
title_element = root.xpath("//xhtml:head/xhtml:title", namespaces=namespaces)[0].text

In [ ]:
title_element

In [ ]:
namespaces = {'xhtml': 'http://www.w3.org/1999/xhtml'}
root = etree.fromstring(data, nsmap=namespaces)

a = root.xpath('//xhtml:section//xhtml:p/text()')[0].text_content()
print(a)  # 出力: hoge

In [ ]:
html = lxml.html.fromstring(content_raw)

In [ ]:
a = html.xpath('//xhtml:section//xhtml:p/text()')[0].text_content()

In [ ]:
a

In [ ]:
base64.b64decode(a).decode()

In [ ]:
title = root.find(etree.XPath("/html/head/title")) 

In [ ]:
# タイトル要素を取得
title = root.find(etree.XPath("/html/head/title"))

# タイトルテキストを出力
print(title.text)

# 各段落要素を取得
paragraphs = root.findall(etree.XPath("/html/body/p"))

# 各段落テキストを出力
for paragraph in paragraphs:
  print(paragraph.text)

In [ ]:
r = response

if r.encoding is None or r.encoding == 'ISO-8859-1':
    r.encoding = r.apparent_encoding

In [ ]:
import base64

In [ ]:
import requests

url = "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/blobs/" + file_info["sha"]

response = requests.get(url)

if response.status_code == 200:
    file_content = base64.b64decode(response.json()["content"]).decode()
    #file_content = response.json()["content"].encode
    print(f"ファイル名: {file_name}")
    print(f"ファイル内容: {file_content}")
else:
    print(f"Error: {response.status_code}")

In [ ]:
for d in file_info_dict.items():
    print(d[1]["content"])
    break

In [ ]:
a = d[1]["content"]

In [ ]:
import lxml

In [ ]:
lxml.html.etree(a)

In [ ]:
import requests
import base64

def get_file_info(url):
  """
  URLを引数にして、ファイルの情報をdictにして返す
  """

  response = requests.get(url)

  if response.status_code == 200:
    # ファイルツリーの情報
    tree_info = response.json()
    file_info_dict = {}
    for file_info in tree_info["tree"]:
      file_name = file_info["path"]
      file_type = file_info["type"]

      # ファイルの内容を取得
      if file_type == "blob":
        url = "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/blobs/" + file_info["sha"]
        response = requests.get(url)
        if response.status_code == 200:
          # ファイルの内容
          file_content = base64.b64decode(response.json()["content"]).decode()
          file_info_dict[file_name] = {
            "type": file_type,
            "content": file_content
          }
        else:
          print(f"Error: {response.status_code}")
      else:
        file_info_dict[file_name] = {
          "type": file_type
        }
    return file_info_dict
  else:
    print(f"Error: {response.status_code}")

# 例
url = "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:src/epub/text"
file_info_dict = get_file_info(url)

# 結果を出力
print(file_info_dict)

In [ ]:
import requests
import base64

def get_file_info(url):
  """
  URLを引数にして、ファイルの情報をdictにして返す
  """

  response = requests.get(url)

  if response.status_code == 200:
    # ファイルツリーの情報
    tree_info = response.json()
    file_info_dict = {}
    for file_info in tree_info["tree"]:
      file_name = file_info["path"]
      file_type = file_info["type"]

      # ファイルの内容を取得
      if file_type == "blob":
        url = "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/blobs/" + file_info["sha"]
        response = requests.get(url)
        if response.status_code == 200:
          # ファイルの内容
          file_content = base64.b64decode(response.json()["content"]).decode()
          file_info_dict[file_name] = {
            "type": file_type,
            "content": file_content
          }
        else:
          print(f"Error: {response.status_code}")
      else:
        file_info_dict[file_name] = {
          "type": file_type
        }
    return file_info_dict
  else:
    print(f"Error: {response.status_code}")

# 例
url = "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:src/epub/text"
file_info_dict = get_file_info(url)

# 結果を出力
print(file_info_dict)

In [ ]:
import requests

url = "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:src/epub/text"

response = requests.get(url)

if response.status_code == 200:
    tree_info = response.json()
    for file_info in tree_info["tree"]:
        file_name = file_info["path"]
        file_type = file_info["type"]
        
        # ファイルの内容を取得
        if file_type == "blob":
            url = "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/blobs/" + file_info["sha"]
            response = requests.get(url)
            if response.status_code == 200:
                # ファイルの内容
                file_content = base64.b64decode(response.json()["content"]).decode()
                print(f"ファイル名: {file_name}")
                print(f"ファイル内容: {file_content}")
            else:
                print(f"Error: {response.status_code}")
        else:
            print(f"ファイル名: {file_name}")
            print(f"ファイルの種類: {file_type}")
else:
    print(f"Error: {response.status_code}")

In [ ]:

import requests

url = "https://api.github.com/repos/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/git/trees/master:src/epub/text"

response = requests.get(url)

if response.status_code == 200:
    # ファイルツリーの情報
    tree_info = response.json()
    for file_info in tree_info["tree"]:
        print(file_info)
else:
    print(f"Error: {response.status_code}")

In [ ]:

import requests

url = "https://github.com/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/tree/master/src/epub/text"

response = requests.get(url)

if response.status_code == 200:
    # ファイルの情報
    file_info = response.json()
    print(file_info)
else:
    print(f"Error: {response.status_code}")

In [ ]:
import subprocess

def curl_hoge():
    result = subprocess.run(["curl", "https://github.com/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/tree/master/src/epub/text"], capture_output=True)
    if result.returncode == 0:
        return json.loads(result.stdout.decode("utf-8"))
    else:
        raise RuntimeError(f"curl hoge failed with code {result.returncode}")

response_dict = curl_hoge()
print(response_dict)

In [ ]:
# URLの完全化
from urllib.parse import urljoin
base_url = "https://standardebooks.org"
page_url = urljoin(base_url, link_url)

In [ ]:
# discription
url = get_url_from_xpath("//div[@class='downloads-container']//a[@property='schema:contentUrl' and contains(@class, 'page')]/@href")

In [ ]:
# URLの完全化
from urllib.parse import urljoin
base_url = "https://standardebooks.org"
#page_url = urljoin(base_url, url)
base_url + "/" + url

In [ ]:
url

In [ ]:
# discription
git_url = get_url_from_xpath('//*[@id="details"]/ul/li[1]/p/a/@href')

In [ ]:
text_list = git_url + "/tree/master/src/epub/text"

In [ ]:
text_list

In [ ]:
# discription
git_url = get_url_from_xpath('//*[@id="details"]/ul/li[1]/p/a')

In [ ]:
response = requests.get('https://github.com/standardebooks/john-maynard-keynes_the-economic-consequences-of-the-peace/tree/master/src/epub/text')
tree = lxml.html.fromstring(response.text.encode('utf-8'))

In [ ]:
link_element = tree.xpath('.//a[@class="Link--primary"]') 

In [ ]:
link_element

In [ ]:
# `a`要素を抽出
link_element = tree.xpath('.//a[@class="Link--primary"]')  

if link_element is not None:
    link = link_element.get('href')
    print("ファイルへのリンク:", link)  
else:
    print("Link was not found")


In [ ]:
# `a`要素を抽出
link_element = tree.xpath('/html/body/div[1]/div[6]/div/main/turbo-frame/div/react-app/div/div/div[1]/div/div/div[2]/div[2]/div/div[3]/div[3]/div/table/tbody/tr[2]')  

In [ ]:
link_element

In [ ]:
# タイトルを取得
title = tree.findtext(".//title")

# 本文を取得
body = tree.find(".//main[@id='content']")

# 処理結果を出力
print(f"タイトル: {title}")
print(f"本文: {body}")

In [ ]:
link = tree.xpath('//*[@id="folder-row-1"]/td[2]/div/div/h3/div/a')

In [ ]:
link

In [ ]:
//*[@id="folder-row-1"]/td[2]/div/div/h3/div/a

In [ ]:
# discription
tree.xpath('/html/body/div[1]/div[6]/div/main/turbo-frame/div/react-app/div/div/div[1]/div/div/div[2]/div[2]/div/div[3]/div[3]/div/table/tbody/tr[2]/td[2]/div/div/h3/div/a/@href') 

In [ ]:
//*[@id="folder-row-1"]/td[2]/div/div/h3/div/a